In [1]:
import pandas as pd
import os
import jieba
import codecs
import jieba
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.scorer import make_scorer
from sklearn import linear_model
from sklearn import metrics
import numpy as np
from time import time 
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

In [2]:
df = pd.read_csv('Train_Data.csv',encoding='utf-8')
df.info()
df.head(10)
#数据概览

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 6 columns):
id            4999 non-null object
title         3660 non-null object
text          4999 non-null object
entity        4981 non-null object
negative      4999 non-null int64
key_entity    2765 non-null object
dtypes: int64(1), object(5)
memory usage: 234.4+ KB


,id,title,text,entity,negative,key_entity
0,83dcefb7,????发表了博文《【富金利】9月富金利教你如何安全提高最大化收益！》网络理财时代参与互联网...,????发表了博文《【富金利】9月富金利教你如何安全提高最大化收益！》网络理财时代参与互联网...,理财时代;富金利,0,NaN
1,1ad5be0d,#小资钱包涉嫌诈骗[超话]##小资钱包[超话]##迫切急待回归活命钱##头号直通中纪委[超话...,#小资钱包涉嫌诈骗[超话]##小资钱包[超话]##迫切急待回归活命钱##头号直通中纪委[超话...,小资钱包;恒丰银行,1,小资钱包;恒丰银行
2,6dd28e9b,?????扫黑除恶 诈骗犯齐聚海淀，保护伞根深蒂固，黑社会嚣张跋扈，出借人走投无路！ 尊敬的...,?????扫黑除恶 诈骗犯齐聚海淀，保护伞根深蒂固，黑社会嚣张跋扈，出借人走投无路！ 尊敬的...,国有投资;资易贷（北京）金融信息服务有限公司;小资钱包;资易贷,1,资易贷（北京）金融信息服务有限公司;小资钱包
3,1db87a14,平安银行车主贷?????? 1??期限长至48期?? 2??滴滴司机可做?? 3??白户逾期...,平安银行车主贷?????? 1??期限长至48期?? 2??滴滴司机可做?? 3??白户逾期...,平安银行;车主贷;平安银行车主贷,0,NaN
4,fa005713,NaN,旺旺贷跑路！深圳警方确定投资人被骗！,旺贷;旺旺贷,1,旺旺贷
5,a15d25e1,NaN,为破解企业发展中的资金周转瓶颈，定南大力推行五个信贷通，截止2018年5月末，全县五个信贷通...,资金周转;小额贷;贷款1;信贷通,0,NaN
6,3854745b,NaN,?????据了解，为控制风险，温州贷在审核方面十分严格，在温州贷进行审核之前，会有资产端合作...,温州贷,0,NaN
7,d137d16e,易融贷可信吗？易融贷安全吗,公司简介 易融贷： 易融贷是由成都易融时代投资咨询有限公司打造的个人借款与出借...,成都易融时代投资咨询有限公司;易融贷,0,NaN
8,3f39b042,宜贷网P2P网贷平台2018/05/25数据研究,宜贷网P2P网贷平台2018/05/25数据研究 2018-05-26???来源：第一网贷...,宜贷网,0,NaN
9,d8819d45,无法相爱的原因很简单，因为我只对你一京东白条提现一勇敢·互助·感恩—京东白条提现—对话宜宾地...,无法相爱的原因很简单，因为我只对你一京东白条提现一勇敢·互助·感恩—京东白条提现—对话宜宾地...,京东白条,0,NaN


In [3]:
import jieba
df['text'] = df.text.apply(lambda x: " ".join(jieba.cut(x)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 1.371 seconds.
Prefix dict has been built successfully.


In [4]:
df = df[['text','negative']]
df.head()

,text,negative
0,? ? ? ? 发表 了 博文 《 【 富 金利 】 9 月富 金利 教 你 如何 安全 提...,0
1,# 小资 钱包 涉嫌 诈骗 [ 超话 ] ## 小资 钱包 [ 超话 ] ## 迫切 急待 ...,1
2,? ? ? ? ? 扫黑 除恶 诈骗犯 齐聚 海淀 ， 保护伞 根深蒂固 ， 黑社会 嚣...,1
3,平安 银行 车主 贷 ? ? ? ? ? ? 1 ? ? 期限 长至 48 期 ? ? ...,0
4,旺旺 贷跑 路 ！ 深圳 警方 确定 投资人 被 骗 ！,1


In [5]:
labels = np.array(df.negative)
df = df[['text']]
training_samples = int(len(labels)*.8)
validation_samples = len(labels)-training_samples

In [6]:
X_train = df[:training_samples]
y_train = labels[:training_samples]
X_valid = df[training_samples: training_samples + validation_samples]
y_valid = labels[training_samples:training_samples + validation_samples]



In [7]:

tfidf = TfidfVectorizer()
X_train  = tfidf.fit_transform(X_train)


In [8]:
classifier=LogisticRegression()
classifier.fit(X_train,y_train)
predictions=classifier.predict(X_test)

e:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Found input variables with inconsistent numbers of samples: [1, 3999]